<a href="https://colab.research.google.com/github/j0gea/BreastCancer/blob/main/BreastCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RSNA Screening Mammography Breast Cancer Detection
https://www.kaggle.com/competitions/rsna-breast-cancer-detection

---

#### **[작성 가이드]**
1. 코드를 수정하실때, 자신의 코드 부분에 마크다운으로 수정하신 **시간, 날짜, 수정사항** 을 기입해주시길 바랍니다.
(수정하신 부분 카톡방에도 짧게 올려주시면 다른 분들도 참고하시고 도움이 될 것 같아요 ㅠㅁㅠ)
2. 모든 코드는 이어지는 형태여야 합니다. 앞 뒤 코드와의 호환성 처리를 부탁드립니다.
3. 만약 새로 추가한 input 데이터가 있다면 이 아래 마크다운에 추가한 데이터의 제목, 링크, 그리고 사용된 파트(1,2,3,4)를 적어주시길 바랍니다.
4. **주석 열심히 달아주세요!!!!**
5. 편집시간이 완전히 겹치면 코드에 오류가 생길 수도 있습니다. 꼭! 먼저 kaggle 노트북에서 테스트를 해보시고 여기는 공유용으로만 사용해주세요.
6. 다른 논의사항은 카톡방에 올려주세요.
7. 참고하신 코드 출처 남겨주시면 감사합니다.




#### **[전체적 수정사항]**
- 2023/11/06 09:17 가이드 기입 및 틀 만들기

(1) [전처리 코드 출처](https://www.kaggle.com/code/theoviel/dicom-resized-png-jpg)


[512링크](https://www.kaggle.com/datasets/theoviel/rsna-breast-cancer-512-pngs)

- 위 노트북을 input으로 넣어주시면 됩니다. (이미 뽑아놓은 이미지) 그렇게 하시면 (1) > 이미지 전환 코드는 사용하지 않으셔도 괜찮습니다.


## 1. 전처리

### (1) 이미지 변환
DICOM -> PNG

*이미지 변환은 시간이 아주 오래걸립니다. 그래서 이미지 변환 코드에 output 데이터가 따로 딸려있는 경우 그것을 대신 input하는 것으로 처리해주시면 아주아주 감사합니다*

In [ ]:
import os
import cv2
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm

# 이미지 변환을 위한 경로 및 변수 설정
SAVE_FOLDER = "ouptut/"
SIZE = 512
EXTENSION = "png"
os.makedirs(SAVE_FOLDER, exist_ok=True)

# 이미지 변환 함수 정의
def process(uid, size=512, save_folder="", extension="png"):
    patient = uid.split('/')[-2]
    image = uid.split('/')[-1][:-4]

    # 이미지 변환을 위해 저장된 데이터 불러오기
    # transformed_data = load_transformed_data(uid)  # 이미지 변환 데이터를 불러오는 함수 사용

    # 아래에 이미지 변환 데이터를 불러오는 코드를 추가합니다.
    transformed_data_path = "path_to_transformed_data_folder"  # 변환된 이미지 데이터 경로
    transformed_data = cv2.imread(os.path.join(transformed_data_path, f"{image}.jpg"))  # 예시로 jpg 이미지로 가정

    # 이미지 전처리 작업 수행
    img = cv2.cvtColor(transformed_data, cv2.COLOR_BGR2RGB)  # 예시로 BGR을 RGB로 변환

    # 이미지 크기 조정
    img = cv2.resize(img, (size, size))

    # 이미지 저장
    cv2.imwrite(save_folder + f"{patient}_{image}.{extension}", img)

# 변환 작업을 병렬로 수행
_ = Parallel(n_jobs=4)(
    delayed(process)(uid, size=SIZE, save_folder=SAVE_FOLDER, extension=EXTENSION)
    for uid in tqdm(train_images[:10])  # train_images는 이미지 경로 리스트로 가정
)
# 2023.11.08  오후 3시 37분

### (2) 이미지 크롭

### (3) 데이터 불균형 해소

코드 수정

2023년 11월 8일: https://www.kaggle.com/code/awsaf49/rsna-bcd-efficientnet-tf-tpu-1vm-infer 에 나온 코드 가져옴 -> 복잡

2023년 11월 9일: https://velog.io/@baes616/Augmentation-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EC%9D%B4%EB%AF%B8%EC%A7%80-%EC%A6%9D%EA%B0%95-%EC%9E%91%EC%97%85google-colab (이미지 증강 파이썬 라이브러리1. Augemntor 기본 구조)



In [ ]:
# 이미지 증강 기본 구조
 import Augmentor


#증강 시킬 이미지 폴더 경로
augmentation_img = Augmentor.Pipeline("이미지 폴더 경로 입")

#인자는 수정하면서 확인하

# 좌우 반전
#augmentation_img.flip_left_right(probability=1.0)


# 상하 반전
#augmentation_img.flip_top_bottom(probability=1.0)


#모퉁이 왜곡
#augmentation_img.skew_corner(probability=1.0)


# 왜곡
# augmentation_img.random_distortion(probability=1, grid_width=10, grid_height=10, magnitude=30)


#확대
#augmentation_img.zoom_random(probability=1, percentage_area=0.8)



# 증강 이미지 수(인자에 원하는 갯수만큼 작성)
img.sample(1)


### (4) 기타 메타데이터 전처리

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedGroupKFold

N_FOLDS = 5
ATEGORY_AUX_TARGETS = ['site_id', 'laterality', 'view', 'implant', 'biopsy', 'invasive', 'BIRADS', 'density', 'difficult_negative_case', 'machine_id', 'age']
TARGET = 'cancer'
ALL_FEAT = [TARGET] + CATEGORY_AUX_TARGETS

RSNA_2022_PATH = '../input/rsna-breast-cancer-detection'
df_train = pd.read_csv(f'{RSNA_2022_PATH}/train.csv')

# k-폴드 5개의 객체 생성
split = StratifiedGroupKFold(N_FOLDS)

# df_train의 'split' 열을 추가 후 각 행에 해당하는 폴드 번호 할당
for k, (_, test_idx) in enumerate(split.split(df_train, df_train.cancer, groups=df_train.patient_id)):
    df_train.loc[test_idx, 'split'] = k

df_train.split = df_train.split.astype(int)
df_train.groupby('split').cancer.mean()

## 2. 머신러닝 (다음주차)

### Resnet50 (기본 비교군)


####(1) Tensorflow

####(2) PyTorch